In [ ]:
## 1. Importing Required Libraries

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from uszipcode import Zipcode, SearchEngine
search = SearchEngine(simple_zipcode=True)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model, metrics 
import statsmodels.api as sm
# from scipy import stats
from sklearn.ensemble import RandomForestRegressor
# from sklearn.datasets import make_regression
from xgboost import XGBRegressor
import lightgbm as lgb
%matplotlib inline

os.chdir("C:\\Users\\Amit Gupta\\Desktop\\Airbnb")

# pd.options.display.max_columns = 5
# pd.options.display.max_rows = 20

In [ ]:
## 2. Importing data and Data cleaning (i.e. handling missing data, etc)

### 2.a) Importing Data

df_train = pd.read_csv("./Data/train10k.csv")
df_train.head()

df_test = pd.read_csv("./Data/test2k.csv")
df_test.head()

In [ ]:
### 2.b) Check for missing values

# Checked for scaling of the variables. There was almost no change in the R2 values of the model after scaling the variables. Therefore, not performing scaling.
# Checking for missing values.
df_missing = df_train.isnull().sum(axis=0).reset_index() # Finding for the count of null values for each of the variable in the dataset.
df_missing.columns = ['column_name', 'missing_count'] # Renaming the column.
df_missing = df_missing.loc[df_missing['missing_count']>0] # Considering or displaying only those columns which have null values.
df_missing = df_missing.sort_values(by='missing_count') # Sorting the variables w.r.t. to the count of null values in the variables (in asc order)
df_missing

In [ ]:
### 2.c) Handling missing values in both train and test dataframe

# Creating a function to replace missing values with zero.
def handleMissingNullValues(dataframes,cols):
    for df in dataframes:
        for col in cols:
            df[col]=df[col].fillna(0)

# Using handleMissingNullValues() function to handle missing values for ["bathrooms","bedrooms","beds","review_scores_rating"] variables
handleMissingNullValues([df_train,df_test],["bathrooms","bedrooms","beds","review_scores_rating"])

In [ ]:
df_train['host_has_profile_pic'] = df_train['host_has_profile_pic'].fillna('f')
df_train['host_identity_verified'] = df_train['host_identity_verified'].fillna('f')
df_train['instant_bookable'] = df_train['instant_bookable'].fillna('f')
df_test['host_has_profile_pic'] = df_test['host_has_profile_pic'].fillna('f')
df_test['host_identity_verified'] = df_test['host_identity_verified'].fillna('f')
df_test['instant_bookable'] = df_test['instant_bookable'].fillna('f')

# Using key value pair to replace values for some variables and then to map it back to the original variable by using inplace=TRUE
cleanup_nums = {"host_has_profile_pic": 
                    {
                        "t": True,
                        "f": False
                    },
                "host_identity_verified": 
                    {
                        "t": True,
                        "f": False
                    },
                "instant_bookable":
                    {
                        "t": True,
                        "f": False                    
                    }
                }
df_train.replace(cleanup_nums, inplace=True)
df_test.replace(cleanup_nums, inplace=True)

In [ ]:
def convertToBoolean(dataframes,cols):
    for df in dataframes:
        for col in cols:
            df[col] = df[col].astype(bool)

convertToBoolean([df_train,df_test],["host_has_profile_pic","host_identity_verified","instant_bookable"])

In [ ]:
# 'neighbourhood'
# Creating new class or group of neighbourhood. Assigning it the value as Unknown
df_train['neighbourhood'] = df_train['neighbourhood'].fillna('Unknown')
df_test['neighbourhood'] = df_test['neighbourhood'].fillna('Unknown')

# 'host_response_rate'
# Removing the % sign from the variable so that we can fill the null values. Replacing % with nothing.
df_train['host_response_rate'] = df_train['host_response_rate'].str.replace('%','')
# Filling the null values with 0.
df_train['host_response_rate'] = df_train['host_response_rate'].fillna('0')
# Converting the variable values into numeric using the pd.to_numeric() function.
df_train['host_response_rate'] = pd.to_numeric(df_train['host_response_rate'])
df_test['host_response_rate'] = df_test['host_response_rate'].str.replace('%','')
df_test['host_response_rate'] = df_test['host_response_rate'].fillna('0')
df_test['host_response_rate'] = pd.to_numeric(df_test['host_response_rate'])

# 'thumbnail_url'
df_train['thumbnail_url'] = df_train['thumbnail_url'].notnull()
df_train['thumbnail_url'] = df_train['thumbnail_url'].astype(bool)
df_test['thumbnail_url'] = df_test['thumbnail_url'].notnull()
df_test['thumbnail_url'] = df_test['thumbnail_url'].astype(bool)

df_train['zipcode'] = df_train['zipcode'].fillna(0)
df_train.loc[df_train.zipcode == ' ', 'zipcode'] = 0
blank_zip = df_train.index[df_train['zipcode']==0].tolist()
for i in blank_zip:
    lat = df_train['latitude'][i]
    lon = df_train['longitude'][i]
    result = np.max(search.by_coordinates(lat, lon, radius=30, returns=5))
    df_train['zipcode'][i]=result.values()[0]    

df_test['zipcode'] = df_test['zipcode'].fillna(0)
df_test.loc[df_test.zipcode == ' ', 'zipcode'] = 0
blank_zip = df_test.index[df_test['zipcode']==0].tolist()
for i in blank_zip:
    lat = df_test['latitude'][i]
    lon = df_test['longitude'][i]
    result = np.max(search.by_coordinates(lat, lon, radius=30, returns=5))
    df_test['zipcode'][i]=result.values()[0]   

In [ ]:
# Again check for missing values in the train dataframe
df_missing = df_train.isnull().sum(axis=0).reset_index()
df_missing.columns = ['column_name', 'missing_count']
df_missing = df_missing.loc[df_missing['missing_count']>0]
df_missing = df_missing.sort_values(by='missing_count')
df_missing

# Again check for missing values in the test dataframe
df_missing_test = df_test.isnull().sum(axis=0).reset_index()
df_missing_test.columns = ['column_name', 'missing_count']
df_missing_test = df_missing_test.loc[df_missing_test['missing_count']>0]
df_missing_test = df_missing_test.sort_values(by='missing_count')
df_missing_test

In [ ]:
# Coverting log prices into normal prices.
df_train['int_price'] = np.exp(df_train['log_price'])
df_train.info()


In [ ]:
## 3. Data Visualization
### 3.a) Plot 1: Integer Pricing per City

plot1 = sns.barplot(x='city',y='int_price',data=df_train)
fig = plot1.get_figure()
fig.savefig('./Plots/Int_Price_Per_City.png')

### 3.b) Plot 2: Average pricing for Property Type w.r.to City

df_train.pivot_table(values='int_price',index='property_type',columns='city') # We will get average prices for each property time for all the cities.

plt.figure(figsize=(20,10))
pivot_df = df_train.pivot_table(values='int_price',index='property_type',columns='city')
plot2 = sns.heatmap(pivot_df, annot=True, cmap="Reds",fmt=".02f")
fig = plot2.get_figure()
fig.savefig('./Plots/AvgPrice_PropType_City.png')

### 3.c) Plot 3: Price variation for Property Type as per Accomodates
df_train.pivot_table(values='int_price',index='property_type',columns='accommodates') 

plt.figure(figsize=(20,10))
pivot_df = df_train.pivot_table(values='int_price',index='property_type',columns='accommodates')
plot3 = sns.heatmap(pivot_df, annot=True, cmap="Reds",fmt=".02f")
fig = plot3.get_figure()
fig.savefig('./Plots/PriceVariation_PropertyType_Accom.png')

### 3.d) Plot 4: Average price w.r.t. number of rooms as per city
df_train.pivot_table(values='int_price',index='bedrooms',columns='city') 

plt.figure(figsize=(20,10))
pivot_df = df_train.pivot_table(values='int_price',index='bedrooms',columns='city')
plot3 = sns.heatmap(pivot_df, annot=True, cmap="Reds",fmt=".02f")
fig = plot3.get_figure()
fig.savefig('./Plots/NoOfRooms_City_AvgPrice.png')

### 3.e) Plot 5: No of reviews per city for every room type
df_train.pivot_table(values='number_of_reviews',index='city',columns='room_type', aggfunc=np.sum) 

plt.figure(figsize=(20,10))
pivot_df = df_train.pivot_table(values='number_of_reviews',index='bedrooms',columns='city',aggfunc=np.sum)
plot3 = sns.heatmap(pivot_df, annot=True, cmap="Reds",fmt=".02f")
fig = plot3.get_figure()
fig.savefig('./Plots/CountReviews_City_RoomType.png')

In [ ]:
# ====================================  Feature Engineering  =========================================

# finding the most recent date in the dataset:
df_train['host_since'] = pd.to_datetime(df_train['host_since'])
recent_host_since = df_train['host_since'].max()
df_train['first_review'] = pd.to_datetime(df_train['first_review'])
recent_first_review = df_train['first_review'].max()
df_train['last_review'] = pd.to_datetime(df_train['last_review'])
recent_last_review = df_train['last_review'].max()
t = []
t.append([recent_host_since,recent_first_review,recent_last_review])
# Therefore, the most recent date in the dataset is "09-12-2017". This date is also the most recent in the testing set.

# Converting most recent date into required format.
import re
def change_date_format(dt):
        return re.sub(r'(\d{4})-(\d{1,2})-(\d{1,2})', '\\3-\\2-\\1', dt)
dt1 = "2017-12-09" 
print("Original date in YYY-MM-DD Format: ",dt1)
print("New date in DD-MM-YYYY Format: ",change_date_format(dt1))
dt1 = change_date_format(dt1) # Getting today's date in dd-mm-yyyy format.
dt1 = pd.to_datetime(dt1)

def convertStringToDays(dataframes,cols,newcols):
    for df in dataframes:
        for index,col in enumerate(cols):
            df[col] =  pd.to_datetime(df[col]) 
            newcolumn= newcols[index]
            df[newcolumn] = dt1 - df[col]
            df[newcolumn] = df[newcolumn].astype('timedelta64[D]')  # Fetching only the number of days from datetime format.
            df[newcolumn] = df[newcolumn].fillna(0)


convertStringToDays([df_train,df_test],["host_since","first_review","last_review"],["host_since_days","days_since_first_review","days_since_last_review"])

df_train.info()
df_test.info()

df_train.to_csv("./Tableau/Data_For_Tableau.csv")

plt.figure(figsize=(20,10))
corr = df_train.corr()
hm = sns.heatmap(round(corr,2),annot=True,cmap="Reds", fmt=".02f")
fig = hm.get_figure()
plt.title("Correlation Matrix")

# We are getting the below variables as correlated with correlation greater than 0.70.
# Zipcode and latitude = -0.87
# zipcode and longitude = -0.99
# Latitude and Longitude = 0.90
# Accomodates and bedrooms = 0.71
# Accomodates and beds = 0.82
# Bedrooms and Beds = 0.71
# Log_price and int_price = 0.84


In [ ]:
# ============================ Working on Train Dataset  =================================================

# Selecting Variables for model
temp_train = df_train.copy()
temp_train.info()

temp_train = temp_train.drop(['description','amenities','name','first_review','host_since','last_review','neighbourhood','int_price'],axis=1)

property_type_dummy = pd.get_dummies(temp_train['property_type'],prefix='property_type')
room_type_dummy = pd.get_dummies(temp_train['room_type'],prefix='room_type')
bed_type_dummy = pd.get_dummies(temp_train['bed_type'],prefix='bed_type')
cancellation_policy_dummy = pd.get_dummies(temp_train['cancellation_policy'],prefix='cancellation_policy')
city_dummy = pd.get_dummies(temp_train['city'],prefix='city')

temp_train = temp_train.drop(['property_type','room_type','bed_type','cancellation_policy','city'],axis=1)
temp_train.info()

temp_train1 = pd.concat([property_type_dummy,room_type_dummy,bed_type_dummy,cancellation_policy_dummy,city_dummy],axis=1)
temp_train1.info()

temp_train2 = pd.concat([temp_train1,temp_train],axis=1)
temp_train2.info()

# Correlation of the Final Merged Dataset with dummies.
plt.figure(figsize=(110,55))
corr = temp_train2.drop(['log_price'],axis=1).corr()
hm = sns.heatmap(round(corr,2),annot=True,cmap="Reds", fmt=".02f")
fig = hm.get_figure()
plt.title("Correlation Matrix")

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=10):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Correlated Value Pairs")
print(get_top_abs_correlations(temp_train2.drop(['log_price'],axis=1),22)) # Getting correlated variable pairs with correlation > 0.50.

top_corr = get_top_abs_correlations(temp_train2.drop(['log_price'],axis=1), 22)
top_corr.to_csv('./Data/top_corr.csv')
# =============================================================================
# Correlated pairs
# =============================================================================
# longitude                     zipcode                       0.988571
# room_type_Entire home/apt     room_type_Private room        0.943475
# city_LA                       latitude                      0.939557
# latitude                      longitude                     0.895887
# latitude                      zipcode                       0.869091
# accommodates                  beds                          0.820039
# city_LA                       longitude                     0.788718
# city_NYC                      zipcode                       0.777360
# city_LA                       zipcode                       0.777015
# city_NYC                      longitude                     0.745928
# property_type_Apartment       property_type_House           0.744267
# bedrooms                      beds                          0.713737
# accommodates                  bedrooms                      0.710387
# city_NYC                      latitude                      0.654099
# bed_type_Futon                bed_type_Real Bed             0.634489
# bathrooms                     bedrooms                      0.585378
# cancellation_policy_flexible  cancellation_policy_strict    0.582007
# city_LA                       city_NYC                      0.578890
# cancellation_policy_moderate  cancellation_policy_strict    0.521276
# number_of_reviews             days_since_first_review       0.512281
# bathrooms                     beds                          0.503537
# bed_type_Pull-out Sofa        bed_type_Real Bed             0.503072
# =============================================================================

# Checking the feature importance
X = temp_train2.drop(['log_price'],axis=1)
y = temp_train2['log_price']

# Calculating the feature importance score using Extra Trees Regressor:
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of Extra Trees Regressor
#plot graph of feature importances for better visualization
plt.figure(figsize=(50,20))
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(68).plot(kind='barh')
plt.show()
feat_importances.to_csv("./Data/importance.csv")

In [ ]:
# ============================ FEATURE REDUCTION  =================================================
# Removing correlated variables to avoid multi-collinearity => therefore removed = ['latitude','longitude','city_SF', 'city_DC', 'city_LA', 'city_NYC', 'city_Chicago', 'city_Boston','beds','accommodates','bedrooms','days_since_last_review','days_since_first_review']

# Removing multicollinearity in the data by removing the variable having less relevant score out of the correlated variables.
# KEEPING ZIPCODE OUT OF ALL OTHER LOCATION PARAMETERS AND ALSO KEEPING THE ROOM_TYPE AND PROPERTY_TYPE DUMMY IN THE MODEL(NOT REMOVING IT)
#temp3 = temp2.drop(['latitude','longitude','city_SF', 'city_DC', 'city_LA', 'city_NYC', 'city_Chicago', 'city_Boston','room_type_Private room','beds','property_type_House','accommodates','bed_type_Futon','days_since_last_review','days_since_first_review'],axis=1)
temp_train3 = temp_train2.drop(['latitude','longitude','city_SF', 'city_DC', 'city_LA', 'city_NYC', 'city_Chicago', 'city_Boston','beds','accommodates','bedrooms','days_since_last_review','days_since_first_review'],axis=1)

temp_train3.info()

final = temp_train3.copy()
# temp_train3 = final.copy()

In [ ]:
# ============================ Working on Train Dataset Complete  =================================================

# ============================ Working on Test Dataset  =================================================
temp_test = df_test.copy()
temp_test.info()

temp_test = temp_test.drop(['description','amenities','name','first_review','host_since','last_review','neighbourhood','host_since','first_review','last_review'],axis=1)

property_type_dummy_test = pd.get_dummies(temp_test['property_type'],prefix='property_type')
room_type_dummy_test = pd.get_dummies(temp_test['room_type'],prefix='room_type')
bed_type_dummy_test = pd.get_dummies(temp_test['bed_type'],prefix='bed_type')
cancellation_policy_dummy_test = pd.get_dummies(temp_test['cancellation_policy'],prefix='cancellation_policy')
city_dummy_test = pd.get_dummies(temp_test['city'],prefix='city')

temp_test = temp_test.drop(['property_type','room_type','bed_type','cancellation_policy','city'],axis=1)
temp_test.info()

temp_test1 = pd.concat([property_type_dummy_test,room_type_dummy_test,bed_type_dummy_test,cancellation_policy_dummy_test,city_dummy_test],axis=1)
temp_test1.info()

temp_test2 = pd.concat([temp_test1,temp_test],axis=1)
temp_test2.info()

temp_test3 = temp_test2.drop(['latitude','longitude','city_SF', 'city_DC', 'city_LA', 'city_NYC', 'city_Chicago', 'city_Boston','beds','accommodates','bedrooms','days_since_last_review','days_since_first_review'],axis=1)
temp_test3.info()

final_test = temp_test3.copy()
# temp_test3 = final_test.copy()

In [ ]:
# ============================ WOrking on Test Dataset Complete =================================================

# ================================ Model Building =============================================

# Making number of columns in test and train equal.
missing = []
# Finding the column names missing in the Test set.
for i in temp_train3.columns:
    if i in temp_test3.columns:
        continue
    else:
        missing.append(i)
        
# Creating new columns for the missing columns in test set and populating their values with zeroes.
feature_difference_df = pd.DataFrame(data=np.zeros((temp_test3.shape[0], len(missing))),
                                     columns=list(missing))
# removing the log_price variable from test set.
feature_difference_df=feature_difference_df.drop(['log_price'],axis=1)
# Joining the missing variables to the test set.
temp_test3 = temp_test3.join(feature_difference_df)
#temp_test3 = temp_test3.drop(['property_type_Yurt'],axis=1)
temp_test3.info()


In [ ]:
# ========================    CASE 1 All variables = Indepdendent variables   =====================================================


# ========================    SPLITING THE TRAIN INTO TRAIN AND VALIDATE     =====================================================

X = temp_train3.drop(['log_price'],axis=1)
y = temp_train3['log_price']

X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.20, random_state=42)

def linearRegressionfunc(X_train,y_train,X_validate):
    # create linear regression object 
    reg = linear_model.LinearRegression() 
    # train the model using the training sets 
    reg.fit(X_train, y_train)
    reg_pred = reg.predict(X_validate) 
    print('The RMSE value for the Linear Regression model is ',"%.4f" % np.sqrt(metrics.mean_squared_error(y_validate,reg_pred)))  # Calculating RMSE for the model
    r_sq = reg.score(X_train, y_train) # Calculating R2 for the model.
    print('Coefficient of determination or R2 of Linear Regression Model is :', "%.4f" % reg.score(X_train, y_train))

def OLSfunc(X_train,y_train):
    est = sm.OLS(y_train, X_train.astype(float)).fit()
    print(est.summary())
    print("The R2 value of the OLS model is ", "%.4f" % est.rsquared)
    return est

# Finding the significant variables out of the total set. WILL USE FOR OTHER MODEL BUILDING LATER.
def findingIndexOfSignificantVariables(est):
    count = 0
    index = -1
    pos = []  # Getting index of variables in X which are not insignificant
    for i in est.pvalues :
        index = index + 1
        if i < 0.05 : 
            count = count + 1
            pos.append(index)
    return pos

# Getting the column names of the significant variables found in OLS.
def gettingColNamesOfSignificantVariables(pos):
    a=[] # Storing column names of significant variables in this variable.
    for index, value in enumerate(X.columns):
        if index in pos:
            a.append(value)
    return a

def randomForestfunc(X_train,y_train,X_validate):
    regr = RandomForestRegressor(max_depth=5, random_state=50)
    regr.fit(X_train, y_train)
    regr_pred = regr.predict(X_validate)
    print('The RMSE value for the RandomForest model is ',"%.4f" % np.sqrt(metrics.mean_squared_error(y_validate,regr_pred)))  # Calculating RMSE for the model
    print('Coefficient of determination or R2 value of RandomForest model is :', "%.4f" % regr.score(X_train, y_train))

def XGBoostfunc(X_train,y_train,X_validate):
    model = XGBRegressor(random_state=50)
    model.fit(X_train, y_train)
    model_pred = model.predict(X_validate)
    print('The RMSE value for the XGBoost model is ',"%.4f" % np.sqrt(metrics.mean_squared_error(y_validate,model_pred)))  # Calculating RMSE for the model
    print('Coefficient of determination or R2 value of XGBoost model is :', "%.4f" % model.score(X_train, y_train))
        
def lightGBMfunc(X_train,y_train,X_validate):
    model = lgb.LGBMRegressor(random_state=50)
    model.fit(X_train,y_train)
    model_pred_light = model.predict(X_validate)
    print('The RMSE value for the LightGBM model is ',"%.4f" % np.sqrt(metrics.mean_squared_error(y_validate,model_pred_light)))  # Calculating RMSE for the model
    print('Coefficient of determination or R2 value of LightGBM model is :', "%.4f" % model.score(X_train, y_train))

linearRegressionfunc(X_train,y_train,X_validate)
est = OLSfunc(X_train,y_train)
pos = findingIndexOfSignificantVariables(est)
significant_colnames = gettingColNamesOfSignificantVariables(pos)
randomForestfunc(X_train,y_train,X_validate)
XGBoostfunc(X_train,y_train,X_validate)
lightGBMfunc(X_train,y_train,X_validate)

In [ ]:
# ========================    CASE 2 Only Significant variables of OLS = Indepdendent variables   =====================================================

new_train_X = X[significant_colnames]
new_train_y =temp_train3["log_price"]

X_train, X_validate, y_train, y_validate = train_test_split(new_train_X, new_train_y, test_size=0.20, random_state=42)

linearRegressionfunc(X_train,y_train,X_validate)
randomForestfunc(X_train, y_train, X_validate)
XGBoostfunc(X_train, y_train, X_validate)
lightGBMfunc(X_train, y_train, X_validate)

In [ ]:
# ========================    Final Model Chosen for Prediction => LightGBM with all variables as independent variables  =====================================================

cols = temp_train3.drop(['log_price'],axis=1).columns
temp_test3 = temp_test3[cols]

X = temp_train3.drop(['log_price'],axis=1)
y = temp_train3['log_price']

X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.20, random_state=42)

# LightGBM
model = lgb.LGBMRegressor(random_state=50)
model.fit(X_train,y_train)
model_pred_light = model.predict(temp_test3)

temp_test3['log_price'] = model_pred_light
df_test['log_price'] = model_pred_light

df_test.to_csv("./Data/Final_Predicted_test.csv")